# Screened Infeasibility UQ: evaluate only

Take the generic DEMO solution, turned into an input file. Remove f-values at iteration vars, and replace their equality constraints with inequalities. Run PROCESS once-through with uncertain inputs, and the response/quantity of interest as the violated constraint residuals, i.e. the infeasibility. This is inexpensive enough to run locally.

Here, the inputs have already been screened using the single-parameter evaluation method to find the most sensitive inputs. This notebook just evaluates the samples and stores them in a database, ready for analysing in in the `infeas_screened_analysis.ipynb` notebook as a later stage.

In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from infeas.decoder import MfileDecoder

## Campaign to capture infeasibility

Using the epistemic uncertain inputs for the entire code, capture the distribution of constraint residuals. To start with, make just 3 inputs uncertain (for running locally).

Begin by defining the parameter bounds and the responses (or quantities of interest).

In [2]:
# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="example_local", work_dir=WORK_DIR)

# Define parameter space
# Uncertainties from Alex's SA paper

params = {
    "fdene": {
        "type": "float",
        "min": 1.1,
        "max": 1.3,
        "default": 1.2,
    },  # check: not sure if this is right var. I think ok
    "hfact": {"type": "float", "min": 1.0, "max": 1.2, "default": 1.2},
    "coreradius": {"type": "float", "min": 0.45, "max": 0.75, "default": 0.75},  # ok
    "fimp_2": {"type": "float", "min": 0.085, "max": 0.115, "default": 0.1},  # ok
    "fimp_14": {
        "type": "float",
        "min": 1.0e-5,
        "max": 1.0e-4,
        "default": 1.0e-5,
    },  # ok
    "psepbqarmax": {"type": "float", "min": 8.7, "max": 9.7, "default": 9.0},  # ok
    "flhthresh": {"type": "float", "min": 0.85, "max": 1.15, "default": 1.15},  # ok
    "cboot": {
        "type": "float",
        "min": 0.95,
        "max": 1.05,
        "default": 1.0,
    },  # ok
    "peakfactrad": {"type": "float", "min": 2.0, "max": 3.5, "default": 3.33},  # ok
    "kappa": {"type": "float", "min": 1.8, "max": 1.9, "default": 1.848},  # ok
    "etaech": {"type": "float", "min": 0.3, "max": 0.5, "default": 0.4},  # ok
    "feffcd": {"type": "float", "min": 0.5, "max": 5.0, "default": 1.0},  # ok
    "etath": {"type": "float", "min": 0.36, "max": 0.4, "default": 0.375},  # ok
    "etaiso": {"type": "float", "min": 0.75, "max": 0.95, "default": 0.9},  # ok
    "boundl_18": {
        "type": "float",
        "min": 3.25,
        "max": 3.75,
        "default": 3.25,
    },  # q^95_min, ok
    "pinjalw": {"type": "float", "min": 51.0, "max": 61.0, "default": 61.0},  # ok
    "alstroh": {"type": "float", "min": 6.0e8, "max": 7.2e8, "default": 6.6e8},  # ok
    "sig_tf_wp_max": {
        "type": "float",
        "min": 5.2e8,
        "max": 6.4e8,
        "default": 6.4e8,
    },  # ok, but might need sig_tf_case_max to be the same too
    "aspect": {"type": "float", "min": 3.0, "max": 3.2, "default": 3.1},
    "boundu_2": {
        "type": "float",
        "min": 11.0,
        "max": 12.0,
        "default": 12.0,
    },  # B_T^max, ok
    "triang": {"type": "float", "min": 0.4, "max": 0.6, "default": 0.5},  # ok
    "vary_param": {
        "type": "string",
        "default": "",
    },  # param being changed: used for analysis only
    "startupratio": {"type": "float", "min": 0.0, "max": 10.0, "default": 1.0}, # HCD redundancy factor
    "step_ucsc_0": {"type": "float", "min": 615.0, "max": 2460.0, "default": 1230.0}, # Nb3Sn unit cost ($/kg)
    "step_ucsc_8": {"type": "float", "min": 1283.5, "max": 5134.0, "default": 2567.0}, # REBCO unit cost ($/kg)
    "step_cconshtf": {"type": "float", "min": 45.5, "max": 182.0, "default": 91.0}, #TF coil steel conduit/sheath ($/m)
    "step_ref_69": {"type": "float", "min": 5.775, "max": 23.1, "default": 11.55}, # ECH HCD unit cost ($/MW)
    "step_ucblbe": {"type": "float", "min": 4200.2, "max": 16800.8, "default": 8400.4}, # Blanket Be unit cost ($/kg)
}

# QoIs
# Violated constraint residuals
qois = [
    "cdirt"
]

Next, define a custom decoder to handle Process's mfile output.

In [3]:
# Create encoder and decoder
# encoder = uq.encoders.GenericEncoder(
#     template_fname="demo_sol_max_net_elec_no_f_IN.template", target_filename="IN.DAT"
# )

encoder = uq.encoders.GenericEncoder(
    template_fname="cost_conf.template", target_filename="cost_conf.toml",
)

decoder = MfileDecoder(target_filename="sol_MFILE.DAT", output_columns=qois)

cmd = "bash /home/jg6173/easyVVUQ-process/cost/cm2/copy_in_dat_and_run_step_cost_models.sh"
actions = uq.actions.local_execute(encoder, cmd, decoder)

# cmd = "process -i IN.DAT"
# actions = uq.actions.local_execute(encoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

In [4]:
# Create PCE sampler
# Vary 3 uncertain inputs
vary = {
    # "startupratio": cp.Uniform(
    #     0.0,
    #     10.0,
    # ),
    # "step_ucsc_0": cp.Uniform(
    #     615.0,
    #     2460.0,
    # ),
    "step_ucsc_8": cp.Uniform(
        1283.5,
        5134.0,
    ),
    # "step_cconshtf": cp.Uniform(
    #     45.5,
    #     182.0,
    # ),
    "step_ref_69": cp.Uniform(
        5.775,
        23.1,
    ),
    "step_ucblbe": cp.Uniform(
        4200.2,
        16800.8,
    ),
    # "step_ucblbreed": cp.Uniform(
    #     401.1,
    #     1604.4,
    # ),
}
# pce_sampler = uq.sampling.PCESampler(vary=vary, polynomial_order=3)

mc_sampler = uq.sampling.MCSampler(vary=vary, n_mc_samples=50)

# Add pce_sampler to campaign
# campaign.set_sampler(pce_sampler)
campaign.set_sampler(mc_sampler)

# Draw samples, execute and collate
campaign.execute().collate(progress_bar=True)
samples = campaign.get_collation_result()

  0%|          | 0/250 [00:00<?, ?it/s]

The IN.DAT file does not contain any obsolete variables.The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.

The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ******************

/home/jg6173/process-gh/process/sctfcoil.py:7271: RuntimeWarning: divide by zero encountered in divide
  plr_vv = ((0.84 / d_vv) * 0.94) * 1e-6
/home/jg6173/process-gh/process/sctfcoil.py:7310: RuntimeWarning: invalid value encountered in scalar divide
  J_vvi = I2 / (2 * numpy.pi * d_vv * Ri_vv)
/home/jg6173/process-gh/process/sctfcoil.py:7271: RuntimeWarning: divide by zero encountered in divide
  plr_vv = ((0.84 / d_vv) * 0.94) * 1e-6
/home/jg6173/process-gh/process/sctfcoil.py:7310: RuntimeWarning: invalid value encountered in scalar divide
  J_vvi = I2 / (2 * numpy.pi * d_vv * Ri_vv)
/home/jg6173/process-gh/process/sctfcoil.py:7271: RuntimeWarning: divide by zero encountered in divide
  plr_vv = ((0.84 / d_vv) * 0.94) * 1e-6
/home/jg6173/process-gh/process/sctfcoil.py:7310: RuntimeWarning: invalid value encountered in scalar divide
  J_vvi = I2 / (2 * numpy.pi * d_vv * Ri_vv)
/home/jg6173/process-gh/process/sctfcoil.py:7271: RuntimeWarning: divide by zero encountered in divide
  p

 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & e

process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARN

...

process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.


 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  
 
 ID  LEVEL  MESSAGE
272     2   Double-null: Upper vertical build forced to match lower                         
  
233     1   CHECK: Joints res not cal. for SC (itart = 1) TF (i_tf_sup = 1),                
  
 39     1   CHECK: Operating with a single null in a double null machine                    
  
242     2   PHYSICS: Bootstrap fraction upper limit enforced                                
  
277     2   CHECK: One or more collision between TF and PF coils. Check PF placement.       
  
 
 ******************************************* End of PROCESS Output ********************************************
 
 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  


process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.


 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  
 
 ID  LEVEL  MESSAGE
272     2   Double-null: Upper vertical build forced to match lower                         
  
233     1   CHECK: Joints res not cal. for SC (itart = 1) TF (i_tf_sup = 1),                
  
 39     1   CHECK: Operating with a single null in a double null machine                    
  
242     2   PHYSICS: Bootstrap fraction upper limit enforced                                
  
277     2   CHECK: One or more collision between TF and PF coils. Check PF placement.       
  
 
 ******************************************* End of PROCESS Output ********************************************
 
 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  


process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
100%|██████████| 250/250 [08:03<00:00,  1.93s/it]
